<a href="https://colab.research.google.com/github/amsath1728-debug/time-series-project-amsath/blob/main/time_series_forcasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# filename: time_series_forecasting.py
"""
Advanced Time Series Forecasting with Attention Mechanisms
Complete implementation with data generation, model training, and evaluation
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout, MultiHeadAttention, LayerNormalization, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

class DataGenerator:
    """Generate complex multivariate time series dataset"""

    def generate_dataset(self, n_obs=2000):
        """Create synthetic dataset with seasonality and volatility"""
        np.random.seed(42)
        dates = pd.date_range(start='2020-01-01', periods=n_obs, freq='H')
        t = np.arange(n_obs)

        # Multiple seasonal patterns
        trend = 0.001 * t + 10 * np.sin(0.0001 * t)
        daily_seasonality = 5 * np.sin(2 * np.pi * t / 24)
        weekly_seasonality = 3 * np.sin(2 * np.pi * t / (24 * 7))

        # Volatility clustering (GARCH-like)
        volatility = np.ones(n_obs)
        returns = np.zeros(n_obs)

        for i in range(1, n_obs):
            volatility[i] = 0.1 + 0.7 * volatility[i-1] + 0.2 * returns[i-1]**2
            returns[i] = np.random.normal(0, np.sqrt(volatility[i]))

        main_series = trend + daily_seasonality + weekly_seasonality + 2 * returns
        series_2 = 0.7 * main_series + 0.3 * np.random.normal(0, 1, n_obs)
        series_3 = 0.5 * main_series + 0.2 * series_2 + 0.3 * np.random.normal(0, 1, n_obs)

        df = pd.DataFrame({
            'timestamp': dates,
            'main_series': main_series,
            'correlated_series_1': series_2,
            'correlated_series_2': series_3,
            'volatility': volatility
        })
        df.set_index('timestamp', inplace=True)
        return df

class FeatureEngineer:
    """Create comprehensive features for time series forecasting"""

    def create_features(self, df, target_col='main_series'):
        """Generate lagged, statistical, and seasonal features"""
        features_df = df.copy()

        # Lagged features
        for lag in [1, 2, 3, 6, 12, 24, 48, 168]:
            features_df[f'lag_{lag}'] = features_df[target_col].shift(lag)

        # Moving statistics
        for window in [24, 168]:
            features_df[f'ma_{window}'] = features_df[target_col].rolling(window).mean()
            features_df[f'std_{window}'] = features_df[target_col].rolling(window).std()

        # Fourier terms for seasonality
        features_df['fourier_sin_daily'] = np.sin(2 * np.pi * features_df.index.hour / 24)
        features_df['fourier_cos_daily'] = np.cos(2 * np.pi * features_df.index.hour / 24)
        features_df['fourier_sin_weekly'] = np.sin(2 * np.pi * features_df.index.dayofweek / 7)
        features_df['fourier_cos_weekly'] = np.cos(2 * np.pi * features_df.index.dayofweek / 7)

        # Time features
        features_df['hour'] = features_df.index.hour
        features_df['day_of_week'] = features_df.index.dayofweek

        return features_df.dropna()

class AttentionLSTMModel(Model):
    """LSTM with self-attention mechanism for time series forecasting"""

    def __init__(self, units=64, attention_heads=4, dropout_rate=0.2, output_steps=1):
        super(AttentionLSTMModel, self).__init__()
        self.units = units
        self.lstm = LSTM(units, return_sequences=True)
        self.attention = MultiHeadAttention(num_heads=attention_heads, key_dim=units)
        self.layer_norm1 = LayerNormalization()
        self.layer_norm2 = LayerNormalization()
        self.ffn = tf.keras.Sequential([
            Dense(units * 2, activation='relu'),
            Dropout(dropout_rate),
            Dense(units)
        ])
        self.global_pool = GlobalAveragePooling1D()
        self.output_layer = Dense(output_steps)
        self.dropout = Dropout(dropout_rate)

    def call(self, inputs, training=False):
        x = self.lstm(inputs)
        attn_output = self.attention(x, x)
        x = self.layer_norm1(x + attn_output)
        ffn_output = self.ffn(x)
        x = self.layer_norm2(x + ffn_output)
        x = self.global_pool(x)
        x = self.dropout(x, training=training)
        return self.output_layer(x)

class StandardLSTMModel:
    """Baseline LSTM model without attention"""

    def create_model(self, input_shape, units=50):
        model = tf.keras.Sequential([
            LSTM(units, return_sequences=True, input_shape=input_shape),
            Dropout(0.2),
            LSTM(units),
            Dropout(0.2),
            Dense(32, activation='relu'),
            Dense(1)
        ])
        return model

class ForecastingPipeline:
    """Complete pipeline for model training and evaluation"""

    def __init__(self):
        self.scaler = StandardScaler()
        self.models = {}
        self.results = {}

    def prepare_sequences(self, features_df, target_col='main_series', seq_length=168):
        """Create sequences for deep learning models"""
        feature_cols = [col for col in features_df.columns if col != target_col]
        X = self.scaler.fit_transform(features_df[feature_cols])
        y = features_df[target_col].values

        X_seq, y_seq = [], []
        for i in range(seq_length, len(X)):
            X_seq.append(X[i-seq_length:i])
            y_seq.append(y[i])

        X_seq = np.array(X_seq)
        y_seq = np.array(y_seq)

        # Train-test split (80-20)
        split_idx = int(0.8 * len(X_seq))
        X_train, X_test = X_seq[:split_idx], X_seq[split_idx:]
        y_train, y_test = y_seq[:split_idx], y_seq[split_idx:]

        # Validation split (10% of training)
        val_split = int(0.9 * len(X_train))
        X_val, y_val = X_train[val_split:], y_train[val_split:]
        X_train, y_train = X_train[:val_split], y_train[:val_split]

        return (X_train, y_train, X_val, y_val, X_test, y_test), feature_cols

    def train_attention_lstm(self, X_train, y_train, X_val, y_val):
        """Train the attention-based model"""
        model = AttentionLSTMModel()
        model.compile(optimizer=Adam(0.001), loss='mse', metrics=['mae'])

        history = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=100,
            batch_size=32,
            verbose=1,
            callbacks=[
                tf.keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True),
                tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=10)
            ]
        )
        self.models['attention_lstm'] = model
        return history

    def train_standard_lstm(self, X_train, y_train, X_val, y_val):
        """Train baseline LSTM model"""
        model = StandardLSTMModel().create_model(X_train.shape[1:])
        model.compile(optimizer=Adam(0.001), loss='mse', metrics=['mae'])

        history = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=100,
            batch_size=32,
            verbose=1,
            callbacks=[tf.keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True)]
        )
        self.models['standard_lstm'] = model
        return history

    def evaluate_models(self, X_test, y_test):
        """Compare model performance"""
        metrics = {}
        for name, model in self.models.items():
            y_pred = model.predict(X_test).flatten()
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            mae = mean_absolute_error(y_test, y_pred)
            mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

            metrics[name] = {'RMSE': rmse, 'MAE': mae, 'MAPE': mape}

        self.results = metrics
        return metrics

def main():
    """Execute complete forecasting pipeline"""
    print("=== Advanced Time Series Forecasting ===")

    # Generate data
    print("1. Generating dataset...")
    data_gen = DataGenerator()
    df = data_gen.generate_dataset(2000)

    # Feature engineering
    print("2. Creating features...")
    feature_engineer = FeatureEngineer()
    features_df = feature_engineer.create_features(df)

    # Prepare pipeline
    print("3. Preparing data sequences...")
    pipeline = ForecastingPipeline()
    (X_train, y_train, X_val, y_val, X_test, y_test), feature_names = pipeline.prepare_sequences(features_df)

    # Train models
    print("4. Training Attention LSTM...")
    pipeline.train_attention_lstm(X_train, y_train, X_val, y_val)

    print("5. Training Standard LSTM...")
    pipeline.train_standard_lstm(X_train, y_train, X_val, y_val)

    # Evaluate
    print("6. Evaluating models...")
    metrics = pipeline.evaluate_models(X_test, y_test)

    # Display results
    print("\n=== MODEL PERFORMANCE ===")
    results_df = pd.DataFrame(metrics).T
    print(results_df)

    return pipeline, metrics, features_df

if __name__ == "__main__":
    pipeline, metrics, features_df = main()

=== Advanced Time Series Forecasting ===
1. Generating dataset...
2. Creating features...
3. Preparing data sequences...
4. Training Attention LSTM...
Epoch 1/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 22s 393ms/step - loss: 19.9814 - mae: 3.6926 - val_loss: 19.5725 - val_mae: 3.6250 - learning_rate: 0.0010
Epoch 2/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 14s 369ms/step - loss: 20.4732 - mae: 3.7498 - val_loss: 15.3278 - val_mae: 3.1483 - learning_rate: 0.0010
Epoch 3/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 14s 375ms/step - loss: 11.9575 - mae: 2.7840 - val_loss: 16.7428 - val_mae: 3.3918 - learning_rate: 0.0010
Epoch 4/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 14s 376ms/step - loss: 7.6658 - mae: 2.1332 - val_loss: 8.4254 - val_mae: 2.3213 - learning_rate: 0.0010
Epoch 5/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 14s 382ms/step - loss: 7.0269 - mae: 2.0817 - val_loss: 4.8741 - val_mae: 1.7576 - learning_rate: 0.0010
Epoch 6/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 15s 386ms/step - loss: 6.1607 - mae: 1.9943 - val_loss: 5.3143 - val_mae: 1.8131 - l